In [94]:
import pandas as pd

# Load Files
Acquisitions = pd.read_csv("./data/acquisitions.csv")
Degrees = pd.read_csv("./data/degrees.csv")
Ipos = pd.read_csv("./data/ipos.csv")
Investments = pd.read_csv("./data/investments.csv")

print("="*60)
print("Dataset 로드 완료!")
print("="*60)

Dataset 로드 완료!


## Acquisitions 데이터 전처리

In [95]:
# 컬럼 제거
Acquisitions_copy = Acquisitions.copy()
Acquisitions_copy = Acquisitions_copy.drop(columns=['id', 'source_url', 'source_description', 'created_at', 'updated_at'])

In [96]:
# acquiring_object_id, acquired_object_id 컬럼명 변경
alter_columns = Acquisitions_copy.rename(columns={
    'acquiring_object_id': 'acquiring_c_id',
    'acquired_object_id': 'acquired_c_id'
}, inplace=True)

In [97]:
# acquired_at 형 변환
Acquisitions_copy['acquired_at'] = pd.to_datetime(Acquisitions_copy['acquired_at'])

# acquired_at 결측값 플래그 컬럼 생성
Acquisitions_copy["is_acquisitions_acq_at_missing"] = Acquisitions_copy["acquired_at"].isna().astype(int)

In [98]:
Acquisitions_copy

,acquisition_id,acquiring_c_id,acquired_c_id,term_code,price_amount,price_currency_code,acquired_at,is_acquisitions_acq_at_missing
0,1,c:11,c:10,NaN,2.000000e+07,USD,2007-05-30,0
1,7,c:59,c:72,cash,6.000000e+07,USD,2007-07-01,0
2,8,c:24,c:132,cash,2.800000e+08,USD,2007-05-01,0
3,9,c:59,c:155,cash,1.000000e+08,USD,2007-06-01,0
4,10,c:212,c:215,cash,2.500000e+07,USD,2007-07-01,0
...,...,...,...,...,...,...,...,...
9557,10525,c:267859,c:285995,stock,2.220000e+08,USD,2013-12-11,0
9558,10526,c:285493,c:286006,NaN,0.000000e+00,USD,2013-12-09,0
9559,10527,c:223792,c:220208,NaN,2.600000e+10,USD,2007-07-03,0
9560,10528,c:227620,c:286176,cash,5.240000e+07,USD,2013-12-12,0


In [99]:
# # csv 쓰기 (데이터 프레임 편집, 분석한 데이터 프레임 저장)
# Acquisitions_copy.to_csv('clean_acquisitions_v1.csv', index=False)
# print("전처리된 데이터를 ames_processed.csv 파일로 저장했습니다.")

전처리된 데이터를 ames_processed.csv 파일로 저장했습니다.


## Degrees 데이터 전처리

In [143]:
# 컬럼 제거
Degrees_copy = Degrees.copy()
Degrees_copy = Degrees_copy.drop(columns=['id', 'graduated_at', 'created_at', 'updated_at'])

In [144]:
# object_id 컬럼명 변경
alter_columns = Degrees_copy.rename(columns={
    'object_id': 'degrees_p_id'
}, inplace=True)

In [145]:
# degree_type 카테고리화
import pandas as pd
import re

def classify_degree(x):
    s = str(x).lower().strip()

    # -------------------------
    # 1) 전문 (associate, 고졸 포함)
    # -------------------------
    if (
        "associate" in s
        or "community college" in s
        or "high school" in s
        or "ged" in s
        or re.search(r"\bhs\b", s)
        or "as" in s
        or "aa" in s
        or s in ["aas", "a.a.s.", "a.s.", "a.a.", "ad", "diploma", "certificate", "vocational", "technical", "tech school", "trade school", "2-year degree", "two-year degree", "sub-degree", "higher diploma", "hnd"]
    ):
        return "Associate degree"

    # -------------------------
    # 2) 학사 (BA/BS/BSc, bachelor 등)
    # -------------------------
    if (
        "bachelor" in s
        or "undergrad" in s
        or re.search(r"\bb\.?a\.?\b", s)   # BA, B.A.
        or re.search(r"\bb\.?s\.?\b", s)   # BS, B.S.
        or "bsc" in s
        or "bcs" in s 
        or "bfa" in s
        or "be" in s
        or "llb" in s
        or "basc" in s
        or "sc.b" in s
        or "sc. b." in s
        or "sc b" in s
        or "s.b" in s
        or "bba" in s
        or "s.b." in s
        or "btech" in s
        or "bba" in s
        or "bcom" in s
        or "scb" in s
        or "sb" in s
        or "bba" in s 
        or "bbs" in s
        or "bid" in s
        or s in ["beng", "b.eng", "bengineering", "baur", "barch", "bva", "bm", "bmus", "bn", "bsn", "bph", "bba", "bcomm", "blaw", "bsw", "b.ed", "bed", "beng(hons)", "bth", "bpharm", "bsee", "bsce", "bsme", "bmet", "bis", "bachlor", "bse"]
    ):
        return "Bachelor’s degree"

    # -------------------------
    # 3) 석사 (master, MS, MSc, MBA 등)
    # -------------------------
    if (
        "master" in s
        or "graduate school" in s
        or "grad school" in s
        or "mba" in s
        or re.search(r"\bm\.?s\.?\b", s)   # MS, M.S.
        or "msc" in s
        or "ma" in s
        or "miage" in s
        or "msba" in s
        or "mphil" in s
        or "llm" in s
        or "mpa" in s
        or "med" in s
        or "mse" in s
        or "meng" in s
        or "eem" in s
        or "mea" in s
        or s in ["mfa", "mim", "mlis", "mm", "mn", "msn", "march", "m.div.", "mha", "mpp", "msw", "macc", "mfin", "m.ed.", "mcm", "mlt", "a.m.", "m.a", "m.a.", "pg"]
    ):
        return "Master’s degree"

    # -------------------------
    # 4) 박사 (PhD, Doctor, MD, JD, EdD, DPhil 등)
    # -------------------------
    if (
        "phd" in s
        or "doctorate" in s
        or "doctoral" in s
        or "doctor of" in s
        or s in ["md", "jd", "edd", "dphil"]
        or "ds" in s
        or "ph d" in s 
        or "ph.d" in s
        or s in ["dba", "psy.d", "psyd", "dpt", "dvm", "aud", "dc", "do", "drph", "scd", "thd", "j.s.d", "s.j.d", "ll.d", "abd", "j.d", "j d", "jd.", "j.d."]
    ):
        return "Doctorate"

    # -------------------------
    # 5) 그 외는 모두 Others
    # -------------------------
    return "Others"

# 새 컬럼 생성
Degrees_copy["cat_degrees_degree_type"] = Degrees_copy["degree_type"].apply(classify_degree)

# 2) categorical로 변환
category_order = ["Associate degree", "Bachelor’s degree", "Master’s degree", "Doctorate", "Others"]

Degrees_copy["cat_degrees_degree_type"] = pd.Categorical(
    Degrees_copy["cat_degrees_degree_type"],
    categories=category_order,
    ordered=True
)

In [146]:
# 매핑 딕셔너리 정의
degree_mapping = {
    "Associate degree": 0,
    "Bachelor’s degree": 1,
    "Master’s degree": 2,
    "Doctorate": 3,
    "Others": 99
}

# 'cat_degrees_degree_type' 컬럼의 값을 매핑
Degrees_copy["degree_level"] = Degrees_copy["cat_degrees_degree_type"].map(degree_mapping)

# degree 결측값 플래그 컬럼 생성
Degrees_copy["is_degree_missing"] = Degrees_copy["degree_type"].isna().astype(int)


In [147]:
# subject 카테고리화

# 1) 매핑 함수 정의
def map_subject(sub):
    s = str(sub).lower()

    # Engineering & Computer Science
    if any(k in s for k in ["computer", "software", "engineering", "information", "data", "robot", "machine", "ai", "electrical"]):
        return "Engineering & Computer Science"
    
    # Natural Science
    if any(k in s for k in ["physics", "chemistry", "biology", "math", "earth", "natural"]):
        return "Natural Science"
    
    # Biotechnology / Medical / Health Science
    if any(k in s for k in ["medicine", "medical", "bio", "neuro", "pharma", "health"]):
        return "Biotechnology / Medical / Health Science"
    
    # Business / Management / Economics
    if any(k in s for k in ["business", "mba", "management", "finance", "economics", "marketing", "account"]):
        return "Business / Management / Economics"
    
    # Social Science
    if any(k in s for k in ["psychology", "sociology", "anthropology", "political", "politics"]):
        return "Social Science"
    
    # Humanities / Arts
    if any(k in s for k in [
    "philosophy", "history", "art", "literature", "humanities",
    "linguistics", "linguistic", "language", 
    "english", "korean", "french", "german", "chinese", "japanese", "spanish",
    "translation", "interpretation", "literary"
    ]):
        return "Humanities / Arts"

    # Law / Public Policy
    if any(k in s for k in ["law", "legal", "policy", "public administration"]):
        return "Law / Public Policy"
    
    # Education
    if any(k in s for k in ["education", "teaching", "pedagogy"]):
        return "Education"
    
    # Design / Architecture
    if any(k in s for k in ["design", "architecture", "architect"]):
        return "Design / Architecture"
    
    return "Others / Unknown"

# 2) 카테고리 매핑 적용
Degrees_copy["cat_degrees_subject"] = Degrees_copy["subject"].apply(map_subject)

# 3) 카테고리 순서 정의
category_order = [
    "Engineering & Computer Science",
    "Natural Science",
    "Biotechnology / Medical / Health Science",
    "Business / Management / Economics",
    "Social Science",
    "Humanities / Arts",
    "Law / Public Policy",
    "Education",
    "Design / Architecture",
    "Others / Unknown"
]

# 4) Categorical 타입으로 변환
Degrees_copy["cat_degrees_subject"] = pd.Categorical(
    Degrees_copy["cat_degrees_subject"],
    categories=category_order,
    ordered=True
)

In [152]:
# institution (학교명) 정규화

def clean_basic(name: str) -> str:
    """기본 텍스트 정리 + 소문자 변환"""
    s = str(name).strip().lower()
    # 이상한 따옴표 통일
    s = s.replace("\x92", "'").replace("’", "'").replace("`", "'")
    # 끝에 붙은 콤마/마침표 제거
    s = re.sub(r"[.,]\s*$", "", s)
    # 여러 공백 하나로
    s = re.sub(r"\s+", " ", s)
    return s

def extract_university_like(s: str) -> str | None:
    """
    university / college 가 들어간 핵심 기관명 추출
    예)
      'university of toronto school of law' → 'university of toronto'
      'boston university, school of management' → 'boston university'
      'babson college of business' → 'babson college'
    """
    # 1) 'university of ...' 패턴
    if "university of " in s:
        m = re.search(
            r"(university of [a-z0-9 .,&'-]+?)(?=\s+school|\s+faculty|\s+department|\s+dept|\s+institute|\s+center|, and\s+| and\s+|/|\(|$)",
            s
        )
        if m:
            return m.group(1).strip()
    
    # 2) '... university' 패턴
    m = re.search(r"([a-z0-9 .,&'-]*?university)", s)
    if m:
        return m.group(1).strip()
    
    # 3) '... college' 패턴
    m = re.search(r"([a-z0-9 .,&'-]*?college)", s)
    if m:
        return m.group(1).strip()
    
    return None

def manual_brand_mapping(s: str) -> str | None:
    """
    university/college 가 명시적이지 않아도
    모두가 아는 브랜드는 본교로 매핑 (같은 애들 → 같은 이름)
    """
    # harvard 계열 (HBS, HLS, Harvard XXX School 등)
    if "harvard" in s:
        return "harvard university"
    # stanford 계열 (GSB 포함)
    if "stanford" in s:
        return "stanford university"
    # wharton → UPenn
    if "wharton" in s:
        return "university of pennsylvania"
    # kellogg → northwestern
    if "kellogg" in s:
        return "northwestern university"
    # ucla anderson → ucla
    if "anderson school" in s and ("ucla" in s or "university of california, los angeles" in s):
        return "university of california, los angeles"
    # UNC Chapel Hill
    if "unc chapel hill" in s:
        return "university of north carolina at chapel hill"
    # MIT 약자만 있는 경우 (혹시 나오면)
    if s.strip() == "mit":
        return "massachusetts institute of technology"
    return None

def flip_university_of(core: str) -> str:
    """
    'university of toronto' → 'toronto university' 로 변환.
    core 는 이미 소문자라고 가정.
    """
    m = re.match(r"university of (.+)", core)
    if m:
        rest = m.group(1).strip()
        rest = re.sub(r"\s+", " ", rest)
        return f"{rest} university"
    return core

def normalize_institution(name: str) -> str:
    """
    최종 정규화:
    - 같은 학교(예: Harvard Business School / Harvard University) → 같은 이름
    - university / college 만 남기고 나머지 꼬리(school, department 등) 제거
    - 'university of X' → 'X university'
    - 전부 소문자
    - london business school, cma ontario 같은 비대학 기관은 그냥 소문자만
    """
    if pd.isna(name) or not str(name).strip():
        return ""
    
    s = clean_basic(name)
    
    # 1) 수동 브랜드 매핑 우선 (harvard business school 등)
    brand = manual_brand_mapping(s)
    if brand is not None:
        core = brand  # 이미 'xxx university' 형태
    else:
        # 2) 일반 university / college 패턴 추출
        core = extract_university_like(s)
    
    if core is not None:
        core = core.strip().lower()
        # 'university of X' 패턴이면 X university로 뒤집기
        if core.startswith("university of "):
            core = flip_university_of(core)
        return core
    
    # university / college 없는 애들 (비즈니스 스쿨, 협회 등)은
    # 그냥 깨끗한 소문자 문자열로만 통일
    return s

# 2. 정규화 적용
Degrees_copy["institution_normalized"] = Degrees_copy["institution"].apply(normalize_institution)


In [155]:
# csv 쓰기 (데이터 프레임 편집, 분석한 데이터 프레임 저장)
Degrees_copy.to_csv('clean_degrees_v3.csv', index=False)
print("전처리된 데이터를 ames_processed.csv 파일로 저장했습니다.")

전처리된 데이터를 ames_processed.csv 파일로 저장했습니다.


## investment 데이터 전처리

In [106]:
# 컬럼 제거
Investments_copy = Investments.copy()
Investments_copy = Investments_copy.drop(columns=['id', 'created_at', 'updated_at'])

In [107]:
# funded_object_id, investor_object_id 컬럼명 변경
alter_columns = Investments_copy.rename(columns={
    'funded_object_id': 'invested_c_id',
    'investor_object_id': 'investor_cfp_id'
}, inplace=True)

In [108]:
Investments_copy

,funding_round_id,invested_c_id,investor_cfp_id
0,1,c:4,f:1
1,1,c:4,f:2
2,3,c:5,f:4
3,4,c:5,f:1
4,4,c:5,f:5
...,...,...,...
80897,57882,c:285244,p:267312
80898,57912,c:60389,p:268570
80899,57929,c:286183,p:365
80900,57929,c:286183,p:176466


In [109]:
# # csv 쓰기 (데이터 프레임 편집, 분석한 데이터 프레임 저장)
# Investments_copy.to_csv('clean_investments_v1.csv', index=False)
# print("전처리된 데이터를 ames_processed.csv 파일로 저장했습니다.")

## ipos 데이터 전처리

In [110]:
# 컬럼 제거
Ipos_copy = Ipos.copy()
Ipos_copy = Ipos_copy.drop(columns=['id', 'source_url', 'source_description', 'created_at', 'updated_at'])

In [111]:
# object_id 컬럼명 변경
alter_columns = Ipos_copy.rename(columns={
    'object_id': 'ipos_c_id'
}, inplace=True)

In [112]:
# public_at 형 변환
Ipos_copy['public_at'] = pd.to_datetime(Ipos_copy['public_at'])

# public_at 결측값 플래그 컬럼 생성
Ipos_copy["is_ipos_public_at_missing"] = Ipos_copy["public_at"].isna().astype(int)

In [113]:
# 1) stock_symbol → exchange 추출
Ipos_copy["exchange"] = Ipos_copy["stock_symbol"].str.split(":").str[0]

# 2) 거래소 티어 매핑 함수
def map_tier(exchange: str) -> str:
    if not isinstance(exchange, str) or exchange.strip() == "":
        return "Unknown"
    ex = exchange.strip().upper()

    # --- Global Tier 1 ---
    global_tier1 = {
        "NASDAQ","NAS","NASDAQGM","NASDAQGS","NASDAQCM","NASDAQ NMS","NASDAQ NMQ",
        "NYSE","NYQ","NYE","NYA",
        "HKEX","HKG","HKSE","HK","HK700",
        "TSE","TYO",        # Tokyo Stock Exchange Main
        "LSE","LON",        # LSE Main
        "KOSPI"
    }
    if ex in global_tier1:
        return "Global_Tier1"

    # --- Growth / Venture ---
    growth_venture = {
        "KOSDAQ",
        "AIM",              # LSE AIM
        "MOTHERS",          # TSE Mothers
        "CHINEXT","STAR","GEM"
    }
    if ex in growth_venture:
        return "Growth_Venture"

    # --- Regional Main ---
    regional_main = {
        "TSX","TSXV",
        "ASX",
        "SGX"
    }
    if ex in regional_main:
        return "Regional_Main"

    # --- OTC / 기타 낮은 인지도 거래소 ---
    otc_other = {
        "OTC","OTCBB","OTCQB","OTCQX","OTCM","OTCMKTS",
        "PINK","GTSM"
    }
    if ex in otc_other:
        return "OTC_Other"

    # 그 외는 Unknown
    return "Unknown"


# 3) 실제 매핑 적용
Ipos_copy["ipo_market_tier"] = Ipos_copy["exchange"].apply(map_tier)


# 4) pd.Categorical 생성
tier_order = ["Global_Tier1", "Growth_Venture", "Regional_Main", "OTC_Other", "Unknown"]

Ipos_copy["cat_ipo_stock_symbol"] = pd.Categorical(
    Ipos_copy["ipo_market_tier"],
    categories=tier_order,
    ordered=True
)

# 5) 변환 컬럼 드롭
Ipos_copy = Ipos_copy.drop(columns=['exchange', 'ipo_market_tier'])

In [127]:
# ipo valuation_amount, valuation_amount nan 값 대체, 비공개 플래그 생성
import numpy as np

# valuation_amount
#0값을 nan으로 대체
Ipos_copy["valuation_amount"] = Ipos_copy["valuation_amount"].replace(0, np.nan)

# 비공개 플래그 변수 생성 → is_ipos_valuation_private
Ipos_copy["is_ipos_valuation_private"] = np.nan
Ipos_copy["is_ipos_valuation_private"] = np.where(# 비공개냐
    Ipos_copy["valuation_amount"].isna(), #비공개(nan)
    1,  #비공개(nan)
    0   #공개(값이 있다)
)
display(Ipos_copy["is_ipos_valuation_private"].value_counts())

is_ipos_valuation_private
1    1151
0     108
Name: count, dtype: int64

In [128]:
# Load Files (경로확인 후 실행!)
usd_aud = pd.read_csv("./환율 계산/USD_AUD_rate.csv")
usd_cad = pd.read_csv("./환율 계산/USD_CAD_rate.csv")
usd_eur = pd.read_csv("./환율 계산/USD_EUR_rate.csv")
usd_gbp = pd.read_csv("./환율 계산/USD_GBP_rate.csv")
usd_jpy = pd.read_csv("./환율 계산/USD_JPY_rate.csv")
usd_sek = pd.read_csv("./환율 계산/USD_SEK_rate.csv")

print("="*60)
print("환율 Dataset 로드 완료!")
print("="*60)

환율 Dataset 로드 완료!


In [129]:
# 각 환율코드별 날짜 min&max 값 찾기 
print("="*60)
print("AUD")
cond1 = (Ipos_copy['valuation_currency_code'] =="AUD")
print(Ipos_copy.loc[cond1, "public_at"].min())
print(Ipos_copy.loc[cond1, "public_at"].max())

print("="*60)
print("CAD")
cond2 = (Ipos_copy['valuation_currency_code'] =="CAD")
print(Ipos_copy.loc[cond2, "public_at"].min())
print(Ipos_copy.loc[cond2, "public_at"].max())

print("="*60)
print("EUR")
cond3 = (Ipos_copy['valuation_currency_code'] =="EUR")
print(Ipos_copy.loc[cond3, "public_at"].min())
print(Ipos_copy.loc[cond3, "public_at"].max())

print("="*60)
print("GBP")
cond4 = (Ipos_copy['valuation_currency_code'] =="GBP")
print(Ipos_copy.loc[cond4, "public_at"].min())
print(Ipos_copy.loc[cond4, "public_at"].max())

print("="*60)
print("JPY")
cond5 = (Ipos_copy['valuation_currency_code'] =="JPY")
print(Ipos_copy.loc[cond5, "public_at"].min())
print(Ipos_copy.loc[cond5, "public_at"].max())

print("="*60)
print("SEK")
cond6 = (Ipos_copy['valuation_currency_code'] =="SEK")
print(Ipos_copy.loc[cond6, "public_at"].min())
print(Ipos_copy.loc[cond6, "public_at"].max())

AUD
NaT
NaT
CAD
1989-06-23 00:00:00
1989-06-23 00:00:00
EUR
2000-02-22 00:00:00
2000-02-22 00:00:00
GBP
2013-05-21 00:00:00
2013-05-21 00:00:00
JPY
1987-06-16 00:00:00
2008-12-17 00:00:00
SEK
NaT
NaT


In [130]:
# 환율 계산 함수
def cal_rate(code):

    if code=="CAD":
        cond2 = (usd_cad["observation_date"]>="1989-06-23") & (usd_cad["observation_date"]<="1989-06-23")
        result = usd_cad.loc[cond2, "DEXCAUS"].mean()

    if code=="EUR":
        cond3 = (usd_eur["observation_date"]>="2000-02-22") & (usd_eur["observation_date"]<="2000-02-22")
        result = usd_eur.loc[cond3, "DEXUSEU"].mean()
    

    if code=="GBP":
        cond4 = (usd_gbp["observation_date"]>="2013-05-21") & (usd_gbp["observation_date"]<="2013-05-21")
        result = usd_gbp.loc[cond4, "DEXUSUK"].mean()

    if code=="JPY":
        cond5 = (usd_jpy["observation_date"]>="1987-06-16") & (usd_jpy["observation_date"]<="2008-12-17")
        result = usd_jpy.loc[cond5, "DEXJPUS"].mean()
    
    return result

In [131]:
# ipo valuation_amount 환율 적용 

cad_mean = cal_rate("CAD")
eur_mean = cal_rate("EUR")
gbp_mean = cal_rate("GBP")
jpy_mean = cal_rate("JPY")
usd_mean = 1.0

Ipos_copy["valuation_amount_usd"] = np.nan
Ipos_copy["valuation_currency_rate"] = np.nan

avg_rate = {
    "CAD": cad_mean,   # 예: 0.85  (U.S. $ to 1 CAD 이면 곱셈)
    "EUR": eur_mean,
    "GBP": gbp_mean,
    "JPY": jpy_mean,   # 예: 110.0 (JPY to 1 USD 이면 나눗셈)
    "USD": 1.0         # 이미 USD인 경우 그대로 사용
}

for cur, r in avg_rate.items():
    cond = Ipos_copy["valuation_currency_code"] == cur
    
    # 방향에 따라 곱하거나 나누기
    if cur in ["CAD", "EUR", "GBP"]:   # "U.S. $ to 1 통화" 타입
        Ipos_copy.loc[cond, "valuation_amount_usd"] = (
            Ipos_copy.loc[cond, "valuation_amount"] * r
        )
        Ipos_copy.loc[cond, "valuation_currency_rate"] = r
    elif cur in ["JPY"]:               # "통화 to 1 U.S. $" 타입
        Ipos_copy.loc[cond, "valuation_amount_usd"] = (
            Ipos_copy.loc[cond, "valuation_amount"] / r
        )
        Ipos_copy.loc[cond, "valuation_currency_rate"] = r
    else:  # USD 등
        Ipos_copy.loc[cond, "valuation_amount_usd"] = Ipos_copy.loc[cond, "valuation_amount"]
        Ipos_copy.loc[cond, "valuation_currency_rate"] = r


#확인
cond1 = (Ipos_copy["valuation_currency_code"] != "USD")
cond2 = (Ipos_copy["valuation_amount"] > 0)
display(Ipos_copy.loc[cond1 & cond2, ["ipos_c_id","valuation_currency_code", "valuation_currency_rate", "valuation_amount", "valuation_amount_usd"]])

,ipos_c_id,valuation_currency_code,valuation_currency_rate,valuation_amount,valuation_amount_usd
67,c:15609,JPY,112.0873,1.089600e+11,9.720994e+08
295,c:9580,EUR,NaN,4.400000e+07,NaN
440,c:9636,JPY,112.0873,1.179000e+05,1.051859e+03
843,c:66694,GBP,1.5170,3.000000e+07,4.551000e+07


In [133]:
# is_ipos_raised_private 실수로 생성 -> drop
# 컬럼 제거
Ipos_copy = Ipos_copy.drop(columns=['is_ipos_raised_private'])

In [135]:
# ipo raised_amount, raised_amount nan 값 대체, 비공개 플래그 생성
import numpy as np

# raised_amount
#0값을 nan으로 대체
Ipos_copy["raised_amount"] = Ipos_copy["raised_amount"].replace(0, np.nan)

# 비공개 플래그 변수 생성 → is_ipos_raised_private
Ipos_copy["is_ipos_raised_private"] = np.nan
Ipos_copy["is_ipos_raised_private"] = np.where(# 비공개냐
    Ipos_copy["raised_amount"].isna(), #비공개(nan)
    1,  #비공개(nan)
    0   #공개(값이 있다)
)
display(Ipos_copy["is_ipos_raised_private"].value_counts())

is_ipos_raised_private
1    1121
0     138
Name: count, dtype: int64

In [136]:
# 각 환율코드별 날짜 min&max 값 찾기 
print("="*60)
print("AUD")
cond1 = (Ipos_copy['raised_currency_code'] =="AUD")
print(Ipos_copy.loc[cond1, "public_at"].min())
print(Ipos_copy.loc[cond1, "public_at"].max())

print("="*60)
print("CAD")
cond2 = (Ipos_copy['raised_currency_code'] =="CAD")
print(Ipos_copy.loc[cond2, "public_at"].min())
print(Ipos_copy.loc[cond2, "public_at"].max())

print("="*60)
print("EUR")
cond3 = (Ipos_copy['raised_currency_code'] =="EUR")
print(Ipos_copy.loc[cond3, "public_at"].min())
print(Ipos_copy.loc[cond3, "public_at"].max())

print("="*60)
print("GBP")
cond4 = (Ipos_copy['raised_currency_code'] =="GBP")
print(Ipos_copy.loc[cond4, "public_at"].min())
print(Ipos_copy.loc[cond4, "public_at"].max())

print("="*60)
print("JPY")
cond5 = (Ipos_copy['raised_currency_code'] =="JPY")
print(Ipos_copy.loc[cond5, "public_at"].min())
print(Ipos_copy.loc[cond5, "public_at"].max())

print("="*60)
print("SEK")
cond6 = (Ipos_copy['raised_currency_code'] =="SEK")
print(Ipos_copy.loc[cond6, "public_at"].min())
print(Ipos_copy.loc[cond6, "public_at"].max())

AUD
NaT
NaT
CAD
2011-02-01 00:00:00
2011-02-01 00:00:00
EUR
2007-06-22 00:00:00
2007-06-22 00:00:00
GBP
2013-05-21 00:00:00
2013-05-21 00:00:00
JPY
1987-06-16 00:00:00
2002-05-23 00:00:00
SEK
NaT
NaT


In [137]:
# 환율 계산 함수
def cal_rate(code):

    if code=="CAD":
        cond2 = (usd_cad["observation_date"]>="2011-02-01") & (usd_cad["observation_date"]<="2011-02-01")
        result = usd_cad.loc[cond2, "DEXCAUS"].mean()

    if code=="EUR":
        cond3 = (usd_eur["observation_date"]>="2007-06-22") & (usd_eur["observation_date"]<="2007-06-22")
        result = usd_eur.loc[cond3, "DEXUSEU"].mean()
    

    if code=="GBP":
        cond4 = (usd_gbp["observation_date"]>="2013-05-21") & (usd_gbp["observation_date"]<="2013-05-21")
        result = usd_gbp.loc[cond4, "DEXUSUK"].mean()

    if code=="JPY":
        cond5 = (usd_jpy["observation_date"]>="1987-06-16") & (usd_jpy["observation_date"]<="2002-05-23")
        result = usd_jpy.loc[cond5, "DEXJPUS"].mean()
    
    return result

In [134]:
Ipos_copy

,ipo_id,ipos_c_id,valuation_amount,valuation_currency_code,raised_amount,raised_currency_code,public_at,stock_symbol,is_ipos_public_at_missing,cat_ipo_stock_symbol,valuation_amount_usd,valuation_currency_rate,is_ipos_valuation_private
0,1,c:1654,NaN,USD,0.000000e+00,USD,1980-12-19,NASDAQ:AAPL,0,Global_Tier1,NaN,1.0,1
1,2,c:1242,NaN,USD,0.000000e+00,NaN,1986-03-13,NASDAQ:MSFT,0,Global_Tier1,NaN,1.0,1
2,3,c:342,NaN,USD,0.000000e+00,NaN,1969-06-09,NYSE:DIS,0,Global_Tier1,NaN,1.0,1
3,4,c:59,NaN,USD,0.000000e+00,NaN,2004-08-25,NASDAQ:GOOG,0,Global_Tier1,NaN,1.0,1
4,5,c:317,1.000000e+11,USD,0.000000e+00,NaN,1997-05-01,NASDAQ:AMZN,0,Global_Tier1,1.000000e+11,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,1372,c:267859,NaN,USD,0.000000e+00,USD,NaT,NYSE:EBS,1,Global_Tier1,NaN,1.0,1
1255,1373,c:39330,NaN,USD,7.250000e+08,USD,2013-12-11,NYSE:ARMK,0,Global_Tier1,NaN,1.0,1
1256,1374,c:220208,NaN,USD,2.350000e+09,USD,2008-12-08,NYSE:HLT,0,Global_Tier1,NaN,1.0,1
1257,1375,c:243568,NaN,USD,0.000000e+00,USD,NaT,OTC:BCLI,1,OTC_Other,NaN,1.0,1


In [139]:
# ipo ipo_raised_amount 환율 적용 

cad_mean = cal_rate("CAD")
eur_mean = cal_rate("EUR")
gbp_mean = cal_rate("GBP")
jpy_mean = cal_rate("JPY")
usd_mean = 1.0

Ipos_copy["ipo_raised_amount_usd"] = np.nan
Ipos_copy["ipo_raised_currency_rate"] = np.nan

avg_rate = {
    "CAD": cad_mean,   # 예: 0.85  (U.S. $ to 1 CAD 이면 곱셈)
    "EUR": eur_mean,
    "GBP": gbp_mean,
    "JPY": jpy_mean,   # 예: 110.0 (JPY to 1 USD 이면 나눗셈)
    "USD": 1.0         # 이미 USD인 경우 그대로 사용
}

for cur, r in avg_rate.items():
    cond = Ipos_copy["raised_currency_code"] == cur
    
    # 방향에 따라 곱하거나 나누기
    if cur in ["CAD", "EUR", "GBP"]:   # "U.S. $ to 1 통화" 타입
        Ipos_copy.loc[cond, "ipo_raised_amount_usd"] = (
            Ipos_copy.loc[cond, "raised_amount"] * r
        )
        Ipos_copy.loc[cond, "ipo_raised_currency_rate"] = r
    elif cur in ["JPY"]:               # "통화 to 1 U.S. $" 타입
        Ipos_copy.loc[cond, "ipo_raised_amount_usd"] = (
            Ipos_copy.loc[cond, "raised_amount"] / r
        )
        Ipos_copy.loc[cond, "ipo_raised_currency_rate"] = r
    else:  # USD 등
        Ipos_copy.loc[cond, "ipo_raised_amount_usd"] = Ipos_copy.loc[cond, "raised_amount"]
        Ipos_copy.loc[cond, "ipo_raised_currency_rate"] = r


#확인
cond1 = (Ipos_copy["raised_currency_code"] != "USD")
cond2 = (Ipos_copy["raised_amount"] > 0)
display(Ipos_copy.loc[cond1 & cond2, ["ipos_c_id","raised_currency_code", "ipo_raised_currency_rate", "raised_amount", "ipo_raised_amount_usd"]])

,ipos_c_id,raised_currency_code,ipo_raised_currency_rate,raised_amount,ipo_raised_amount_usd
15,c:108,EUR,1.3438,2.541110e+05,3.414744e+05
29,c:1863,EUR,1.3438,3.570000e+07,4.797366e+07
843,c:66694,GBP,1.5170,3.880000e+06,5.885960e+06
998,c:141691,CAD,0.9921,1.130000e+07,1.121073e+07
1200,c:277665,JPY,NaN,1.100000e+09,NaN
1230,c:27295,GBP,1.5170,2.440000e+10,3.701480e+10


In [140]:
Ipos_copy

,ipo_id,ipos_c_id,valuation_amount,valuation_currency_code,raised_amount,raised_currency_code,public_at,stock_symbol,is_ipos_public_at_missing,cat_ipo_stock_symbol,valuation_amount_usd,valuation_currency_rate,is_ipos_valuation_private,is_ipos_raised_private,ipo_raised_amount_usd,ipo_raised_currency_rate
0,1,c:1654,NaN,USD,NaN,USD,1980-12-19,NASDAQ:AAPL,0,Global_Tier1,NaN,1.0,1,1,NaN,1.0
1,2,c:1242,NaN,USD,NaN,NaN,1986-03-13,NASDAQ:MSFT,0,Global_Tier1,NaN,1.0,1,1,NaN,NaN
2,3,c:342,NaN,USD,NaN,NaN,1969-06-09,NYSE:DIS,0,Global_Tier1,NaN,1.0,1,1,NaN,NaN
3,4,c:59,NaN,USD,NaN,NaN,2004-08-25,NASDAQ:GOOG,0,Global_Tier1,NaN,1.0,1,1,NaN,NaN
4,5,c:317,1.000000e+11,USD,NaN,NaN,1997-05-01,NASDAQ:AMZN,0,Global_Tier1,1.000000e+11,1.0,0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,1372,c:267859,NaN,USD,NaN,USD,NaT,NYSE:EBS,1,Global_Tier1,NaN,1.0,1,1,NaN,1.0
1255,1373,c:39330,NaN,USD,7.250000e+08,USD,2013-12-11,NYSE:ARMK,0,Global_Tier1,NaN,1.0,1,0,7.250000e+08,1.0
1256,1374,c:220208,NaN,USD,2.350000e+09,USD,2008-12-08,NYSE:HLT,0,Global_Tier1,NaN,1.0,1,0,2.350000e+09,1.0
1257,1375,c:243568,NaN,USD,NaN,USD,NaT,OTC:BCLI,1,OTC_Other,NaN,1.0,1,1,NaN,1.0


In [142]:
# csv 쓰기 (데이터 프레임 편집, 분석한 데이터 프레임 저장)
Ipos_copy.to_csv('clean_ipos_v2.csv', index=False)
print("전처리된 데이터를 ames_processed.csv 파일로 저장했습니다.")

전처리된 데이터를 ames_processed.csv 파일로 저장했습니다.
